In [1]:
import TunaSims
import numpy as np
from funcOb import func_ob
import pandas as pd
import tools_fast

In [2]:
def harmonic_mean_distance(p, q):
    r"""
    Harmonic mean distance:

    .. math::

        1-2\sum(\frac{P_{i}Q_{i}}{P_{i}+Q_{i}})
    """
    return 2 * np.sum(p * q / (p + q))

def lorentzian_distance(p, q):
    r"""
    Lorentzian distance:

    .. math::

        \sum{\ln(1+|P_i-Q_i|)}
    """
    return np.sum(np.log(1 + np.abs(p - q)))

def matusita_distance(p, q):
    r"""
    Matusita distance:

    .. math::

        \sqrt{\sum(\sqrt{P_{i}}-\sqrt{Q_{i}})^2}
    """
    return np.sum(np.power(np.sqrt(p) - np.sqrt(q), 2))

def probabilistic_symmetric_chi_squared_distance(p, q):
    r"""
    Probabilistic symmetric χ2 distance:

    .. math::

        \frac{1}{2} \times \sum\frac{(P_{i}-Q_{i}\ )^2}{P_{i}+Q_{i}\ }
    """
    return 1 / 2 * np.sum(np.power(p - q, 2) / (p + q))

In [3]:
query = np.array([[1,10],[2,50]])
target = np.array([[1,20],[2,40]])

In [4]:
harmonic_mean = TunaSims.ExpandedTuna(query_intensity_a = 1,
                                  target_intensity_a = 1,
                                  sigmoid_score = False,
                                  mult_a = 2,
                                  add_norm_b = 1)

prob = TunaSims.ExpandedTuna(query_intensity_a = 1,
                                  target_intensity_a = 1,
                                  sigmoid_score = False,
                                  dif_a = 1/2,
                                  dif_b = 2,
                                  add_norm_b = 1)

In [5]:
print(f'harmonic_mean: {harmonic_mean.predict(query, target) - harmonic_mean_distance(query[:,1], target[:,1])}')
print(f'Chisquare: {prob.predict(query, target) - probabilistic_symmetric_chi_squared_distance(query[:,1], target[:,1])}')

hi
hi
harmonic_mean: -2.9669867629422697e-06
hi
hi
Chisquare: 1.0596381283534129e-07


In [6]:
init_vals = {
    'query_intensity_a': 1.0,
    'target_intensity_a': 1.0,
    'mult_a' : 1,
    'add_norm_b' : 1
}

fixed_vals = {'sigmoid_score' : False}
bounds = {'add_norm_b': (0, np.inf)}

testerooni = func_ob('teesterooni',
                     sim_func = TunaSims.ExpandedTuna,
                     init_vals = init_vals,
                     fixed_vals = fixed_vals,
                     max_iter = 100,
                     lambdas = 0.0000001)

print(testerooni.sim_func.target_intensity_b)


0


In [7]:
demo_matches = pd.read_pickle('/Users/jonahpoczobutt/projects/TunaRes/metlinGnps_NIST20_matchedPol/intermediateOutputs/splitMatches/train/10_ppm/chunk_1.pkl')
#demo_matches['score'] = 1 - demo_matches['InchiCoreMatch']

In [8]:
matched_scores = list()
for i in range(len(demo_matches)):
    
    matched = tools_fast.match_spectrum(demo_matches.iloc[i]['query'], demo_matches.iloc[i]['target'], ms2_da = 0.05)
    matched_scores.append(harmonic_mean_distance(matched[:,1], matched[:,2]))


demo_matches['score'] = matched_scores


In [9]:
testerooni.fit(demo_matches, verbose = 10000)

hi
hi


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
demo_matches[demo_matches['score'] == 1811.643310546875]

In [ ]:
testerooni.sim_func.grads1_agg_int['mult_a']

In [ ]:
testerooni.converged

In [ ]:
testerooni.sim_func.add_norm_b

In [ ]:
testerooni.lambdas

In [ ]:
min(max(1,2),3)